In [2]:
import cv2
import numpy as np

net = cv2.dnn.readNetFromONNX("best.onnx")
file = open("coco.txt", "r")
classes = file.read().split('\n')
print(classes)

image_paths = ["../Major P/set4dataset-62(410411132780525).jpg"]  # Add the paths of the images you want to process

for image_path in image_paths:
    img = cv2.imread(image_path)

    img_height, img_width = img.shape[:2]  # Get the height and width of the image

    blob = cv2.dnn.blobFromImage(img, scalefactor=1/255, size=(640, 640), mean=[0, 0, 0], swapRB=True, crop=False)
    net.setInput(blob)
    detections = net.forward()[0]

    # cx, cy, w, h, confidence, 80 class_scores
    # class_ids, confidences, boxes

    class_ids = []
    confidences = []
    boxes = []
    rows = detections.shape[0]

    x_scale = img_width / 640
    y_scale = img_height / 640

    for i in range(rows):
        row = detections[i]
        confidence = row[4]
        if confidence > 0.5:
            class_scores = row[5:]
            ind = np.argmax(class_scores)
            if class_scores[ind] > 0.5:
                class_ids.append(ind)
                confidences.append(confidence)
                cx, cy, w, h = row[:4]
                x1 = int((cx - w / 2) * x_scale)
                y1 = int((cy - h / 2) * y_scale)
                width = int(w * x_scale)
                height = int(h * y_scale)
                box = np.array([x1, y1, width, height])
                boxes.append(box)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.5)

    for i in indices:
        x1, y1, w, h = boxes[i]
        label = classes[class_ids[i]]
        conf = confidences[i]
        text = label + " {:.2f}".format(conf)

        # Crop the detection region
        crop_img = img[y1:y1+h, x1:x1+w]

        # Save cropped image with the respective class name
        save_path = f"{label}.jpg"
        cv2.imwrite(save_path, crop_img)

        cv2.rectangle(img, (x1, y1), (x1 + w, y1 + h), (255, 0, 0), 2)
        cv2.putText(img, text, (x1, y1 - 2), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 0, 255), 2)

    cv2.imwrite('my.jpg', img)
    cv2.imshow("Image", img)
    cv2.waitKey(0)

cv2.destroyAllWindows()

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\onnx\onnx_importer.cpp:274: error: (-5:Bad argument) Can't read ONNX file: best.onnx in function 'cv::dnn::dnn4_v20230620::ONNXImporter::ONNXImporter'


In [1]:
!cd yolov5